In [4]:
import os
import mne
import numpy as np
import pandas as pd
import glob

# Exploring CHB-MIT dataset

## Reading edf files

In [3]:
patient_id = 1
DATASET_PATH = os.path.join(os.getcwd(), 'data', 'chb01-summary.txt') #ONLY FOR TESTING
patient_id_str = str(patient_id).zfill(2)

# all_file_path = glob(f'chb{patient_id_str}/*.edf')
# print(len(all_file_path))

file_path = os.path.join('data', 'chb01', 'chb01_03.edf')

In [4]:
# raw = mne.io.read_raw_edf(all_file_path[0])
raw = mne.io.read_raw_edf(file_path)
print(raw.get_channel_types())
print(raw.info)
print(raw.ch_names)
print(raw.get_data().shape)

Extracting EDF parameters from /home/guisoares/soares_repo/SCC0276-Machine-Learning/chb01/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg']
<Info | 7 non-empty values
 bads: []
 ch_names: FP1-F7, F7-T7, T7-P7, P7-O1, FP1-F3, F3-C3, C3-P3, P3-O1, ...
 chs: 23 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 128.0 Hz
 meas_date: 2076-11-06 13:43:04 UTC
 nchan: 23
 projs: []
 sfreq: 256.0 Hz
>
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8-0', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8-1']


/tmp/ipykernel_21769/427083036.py:2: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path)


(23, 921600)


In [3]:
# def read_data_edf(file_path):
#     data = mne.io.read_raw_edf(file_path, preload=True)
#     data.set_eeg_reference()
#     data.filter(l_freq=0.5, h_freq=45)
#     epochs = mne.make_fixed_length_epochs(data, duration=10, overlap=1)
#     epochs_array = epochs.get_data()
#     return epochs, epochs_array

def read_edf_to_raw(file_path):
    raw = mne.io.read_raw_edf(file_path, preload=True)
    raw.set_eeg_reference()
    raw.filter(l_freq=0.5, h_freq=45)
    return raw

## Extract features

In [2]:
from utils import pyeeg
from scipy.stats import kurtosis, skew
from scipy.signal import argrelextrema, welch
from scipy.integrate import cumtrapz
import statistics 

feature_names = ["rms", "variance", "kurtosis", "skewness", "max_amp", "min_amp", "n_peaks", "n_crossings", 
        "hfd", "pfd", "hurst_exp", "spectral_entropy", "total_power", "median_freq", "peak_freq", 
        "hjorth_mobility", "hjorth_complexity", "power_1hz", "power_5hz", "power_10hz", "power_15hz", "power_20hz"]

def eeg_features(data):
    data = np.asarray(data)
    res  = np.zeros([22])
    Kmax = 5
    # M    = 10
    # R    = 0.3
    Band = [1,5,10,15,20,25]
    Fs   = 256
    power, power_ratio = pyeeg.bin_power(data, Band, Fs)
    f, P = welch(data, fs=Fs, window='hanning', noverlap=0, nfft=int(256.))       # Signal power spectrum
    area_freq = cumtrapz(P, f, initial=0)
    res[0] = np.sqrt(np.sum(np.power(data, 2)) / data.shape[0])                   # amplitude RMS
    res[1] = statistics.stdev(data)**2                                            # variance
    res[2] = kurtosis(data)                                                       # kurtosis
    res[3] = skew(data)                                                           # skewness
    res[4] = max(data)                                                            # max amplitude
    res[5] = min(data)                                                            # min amplitude
    res[6] = len(argrelextrema(data, np.greater)[0])                              # number of local extrema or peaks
    res[7] = ((data[:-1] * data[1:]) < 0).sum()                                   # number of zero crossings
    res[8] = pyeeg.hfd(data, Kmax)                                                # Higuchi Fractal Dimension
    res[9] = pyeeg.pfd(data)                                                      # Petrosian Fractal Dimension
    res[10] = pyeeg.hurst(data)                                                   # Hurst exponent
    res[11] = pyeeg.spectral_entropy(data, Band, Fs, Power_Ratio=power_ratio)     # spectral entropy (1.21s)
    res[12] = area_freq[-1]                                                       # total power
    res[13] = f[np.where(area_freq >= res[12] / 2)[0][0]]                         # median frequency
    res[14] = f[np.argmax(P)]                                                     # peak frequency
    res[15], res[16] = pyeeg.hjorth(data)                                         # Hjorth mobility and complexity
    res[17] = power_ratio[0]
    res[18] = power_ratio[1]
    res[19] = power_ratio[2]
    res[20] = power_ratio[3]
    res[21] = power_ratio[4]
    # res[22] = pyeeg.samp_entropy(data, M, R)             # sample entropy
    # res[23] = pyeeg.ap_entropy(data, M, R)             # approximate entropy (1.14s)
    return (res)

### Test features

In [7]:
# read raw
raw = read_edf_to_raw(file_path)

start, stop = raw.time_as_index([10, 100])
epoch = raw[:, start:stop][0]
print(epoch.shape)

features = []
for eletrode in epoch :
    features.append(eeg_features(eletrode))
    
features_array = np.array(features).ravel()
print(features_array.shape)

Extracting EDF parameters from /home/guisoares/soares_repo/SCC0276-Machine-Learning/chb01/chb01_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_21769/276806641.py:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)

(23, 23040)


/home/guisoares/soares_repo/SCC0276-Machine-Learning/pyeeg.py:738: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (p, _, _, _) = numpy.linalg.lstsq(x, L)
/home/guisoares/soares_repo/SCC0276-Machine-Learning/pyeeg.py:819: RuntimeWarning: invalid value encountered in true_divide
  R_S = R_T / S_T
/home/guisoares/soares_repo/SCC0276-Machine-Learning/pyeeg.py:823: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  [m, c] = numpy.linalg.lstsq(A, R_S)[0]


(506,)


Now we understood how the dataset is organized and how to extract features along the epochs. Now lets develop the loop that will take the data, extract features and save with a better understanding way to analyse and work with. Also, it will be able to load the saved data to train and evaluate our models

## Visualize features per channel and per epoch

In [ ]:
# get features of specific channel
def features_by_channel(features_array):
    features_channel = []
    MAX_CHANNELS = 23
    for i in range(MAX_CHANNELS):
        array = features_array[i*22:(i+1)*22];
        features_channel.append(pd.DataFrame(array.T, columns=feature_names))
    return features_channel

features_channel = features_by_channel(features_array)
print('Info for channel 23: ')

# Shape of features_channel=(23, 1596, 12)
features_channel[22].shape

## Labels

Analysing the summary.txt of each patient we could annotate the files with seizures

In [6]:
## READ SUMMARY (TODO: automate this)
seizures_dict = {"chb01_03": [[2996, 3036]],
                "chb01_04": [[1467, 1494]],
                "chb01_15": [[1732, 1772]],
                "chb01_16": [[1015, 1066]],
                "chb01_18": [[1720, 1810]],
                "chb01_21": [[327, 420]],
                "chb01_26": [[1862, 1963]],
                
                "chb02_16": [[130, 212]],

                "chb05_06": [[417, 532]], 
                "chb05_13": [[1086, 1196]],
                "chb05_16": [[2317, 2413]], 
                "chb05_17": [[2451, 2571]],
                "chb05_22": [[2348, 2465]],
                
                "chb08_02": [[2670, 2841]], 
                "chb08_05": [[2856, 3046]],
                "chb08_11": [[2988, 3211]], 
                "chb08_13": [[2417, 2577]],
                "chb08_21": [[2083, 2347]]}

# Preprocessing and labelling data

In [ ]:
# # patient_folder = Path.cwd()/'chbmit'/'chb{}'.format(str(patient_id).zfill(2))

# patient_id = 1
# patient_id_str = str(patient_id).zfill(2)

# patient_folder = os.path.join(DATASET_PATH, f'chb{patient_id_str}')
# print(f"Serching for patient {patient_id_str} files at {patient_folder} ...")
# patient_files = glob(os.path.join(patient_folder, f'*.edf'))
# print(f"Found {len(patient_files)} files")
# summary_path = os.path.join(patient_folder, f'chb{patient_id_str}-summary.txt')

In [9]:

################ INPUT YOUR DATASET PATH HERE
DATASET_PATH = os.path.join(os.getcwd(), 'data') #ONLY FOR TESTING

################ CHOOSE THE FILES THAT YOU WANT HERE
selected_files = {'chb01': ['01','02','03','04','15','16','18','21','26']}
patient_files = [os.path.join(DATASET_PATH, folder, f"{folder}_{fn}.edf") for folder, fn_list in selected_files.items() for fn in fn_list]

for patient_file in patient_files:
    print(patient_file)


# parameters for epochs generation
curr_time = 0
epoch_time = 10
overlap = 5

# Divide into epochs
dataset = []
for file_path in patient_files:
    # get filename: chbxx_xx.edf
    filename = os.path.split(file_path)[1]

    # remove .edf staying only chbxx_xx
    filename = os.path.splitext(filename)[0]
    
    # read raw
    raw = read_edf_to_raw(file_path)

    curr_time = 0
    while curr_time <= max(raw.times) + 0.01 - epoch_time:  # max(raw.times) = 3600
        
        # temporary list with features of current epoch
        epoch_features = []

        # calculate window and get data to epoch array
        start_time = curr_time 
        if start_time < 0.:
            start_time = 0.
        end_time = curr_time + epoch_time
        start, stop = raw.time_as_index([start_time, end_time])
        epoch = raw[:, start:stop][0]

        # start time as ID
        epoch_features.extend([start_time])
        
        # extract features
        for channel in epoch:
            epoch_features.extend(eeg_features(channel))

        # seizure flag for y
        aux = []
        if filename in seizures_dict:  # if file has seizure
            for seizure in seizures_dict[filename]:
                if start_time > seizure[0] and start_time < seizure[1]:
                    aux.append(1)
                elif start_time + epoch_time > seizure[0] and start_time + epoch_time < seizure[1]:
                    aux.append(1)
                else:
                    aux.append(0)
        else:    
            aux.append(0)

        # if the current time is inside at least one seizure interval -> 1; otherwise -> 0
        if 1 in aux:
            epoch_features.extend([1])
        else:
            epoch_features.extend([0])

        # append apoch features and label to the dataset
        dataset.append(epoch_features)

        # calculate next current time
        curr_time = curr_time + epoch_time - overlap  
        print("Section ", str(len(dataset)), "; start: ", start, " ; stop: ", stop)
    
    # generate columns labels
    column_names = []
    for channel in raw.ch_names:
        for name in feature_names:
            column_names.append(channel + "_" + name)
    column_names.append("seizure")

    res = pd.DataFrame(dataset, columns=column_names)
    res.to_csv(os.path.join("processed_data", filename + '.csv'), index=False) 

/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_01.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_02.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_03.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_04.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_15.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_16.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_18.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_21.edf
/home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_26.edf
Extracting EDF parameters from /home/guisoares/soares_repo/SCC0276-Machine-Learning/data/chb01/chb01_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...


/tmp/ipykernel_25870/276806641.py:10: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw = mne.io.read_raw_edf(file_path, preload=True)


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 1691 samples (6.605 sec)



/home/guisoares/soares_repo/SCC0276-Machine-Learning/utils/pyeeg.py:738: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  (p, _, _, _) = numpy.linalg.lstsq(x, L)
/home/guisoares/soares_repo/SCC0276-Machine-Learning/utils/pyeeg.py:819: RuntimeWarning: invalid value encountered in true_divide
  R_S = R_T / S_T
/home/guisoares/soares_repo/SCC0276-Machine-Learning/utils/pyeeg.py:823: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  [m, c] = numpy.linalg.lstsq(A, R_S)[0]


KeyboardInterrupt: 

In [26]:
dataset = np.array(dataset)
dataset.shape

(16, 507)

After that we could generate the data with the csv extension. Each edf file has it's representant .csv file with the feature extraction.